In [4]:
%pip install langchain_openai

  Using cached langchain_openai-0.1.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_core-0.1.39-py3-none-any.whl.metadata (5.9 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.1.39-py3-none-any.whl.metadata (13 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pydantic-2.6.4-py3-none-any.whl.metadata (85 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using 

In [7]:
from langchain_openai import ChatOpenAI

In [6]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)


In [8]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks that can be used to streamline the testing process. It can also provide support for different types of testing, such as unit testing, integration testing, and end-to-end testing. Additionally, Langsmith can help with generating test data, managing test cases, and analyzing test results to identify any issues or errors in the code. Overall, Langsmith can help improve the efficiency and effectiveness of testing activities in software development projects.', response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 15, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-7cacefe2-2b91-4459-b5b7-0e54e57d6b8d-0')

In [9]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm 

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith provides tools and libraries for creating automated tests for your codebase. This can help you ensure that your code functions as expected and that any changes you make do not introduce new bugs.\n\n2. Test Scripting: Langsmith allows you to write test scripts in a variety of programming languages, making it easy to create and run tests for different parts of your application.\n\n3. Test Data Generation: Langsmith includes features for generating test data, which can be useful for testing edge cases and ensuring that your code handles a variety of inputs correctly.\n\n4. Integration Testing: Langsmith can be used to perform integration testing, allowing you to test how different components of your application work together.\n\n5. Performance Testing: Langsmith offers tools for performance testing, helping you identify bottlenecks in your code and optimize its performance.\n\nOverall, 

In [ ]:
LANGCHAIN_API_KEY = "ls__c1506344a66146b2b16a690143aafd21"

In [14]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("./rate.pdf")
# pages = loader.load_and_split()

from langchain_community.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("./rate.pdf")
data = loader.load()

[nltk_data] Downloading package punkt to /Users/ivorytang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ivorytang/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)
vector = FAISS.from_documents(documents, embeddings)

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [22]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [23]:
response = retrieval_chain.invoke({"input": "if i have a 720 fico score and want a 30 year loan, what's the adjusted rate?"})
print(response["answer"])


Based on the information provided, for a 720 FICO score, the adjusted rate for a 30-year loan would be 6.125%.


In [41]:
%pip install crewai
%pip install 'crewai[tools]'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [42]:
from crewai import Agent
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

In [43]:
# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

In [45]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [15]:
from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential  # Optional: Sequential task execution is default
)

In [46]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
print(result)

 [DEBUG]: == Working Agent: Ratesheet Analyzer
 [INFO]: == Starting Task: Find 30 year conventional loan rate with a 15 day lock for a person with a 700 credit score and $100,000 annual income with no debts, looking for a single family home in San Francisco with loan-to-value of 20%.


> Entering new CrewAgentExecutor chain...
I need to find the interest rate for a 30 year conventional loan with a 15-day lock for a borrower with a 700 credit score and $100,000 annual income with no debts. The property is a single-family home in San Francisco and the loan-to-value is 20%. The best way to find this information would be to search the Rocket Mortgage.pdf document for terms related to these criteria.

Action: 
Search a PDF's content

Action Input: 
{
"search_query": "30 year conventional loan 15 day lock 700 credit score $100,000 annual income no debts single family home San Francisco 20% loan-to-value"
}
 

Relevant Content:
Rocket Credit Title Credit (HomePath properties when Fannie Mae p

In [22]:
from crewai import Crew, Process, Agent, Task

# Define your agents
researcher = Agent(
  role='Researcher',
  goal='Conduct foundational research',
  backstory='An experienced researcher with a passion for uncovering insights'
)
analyst = Agent(
  role='Data Analyst',
  goal='Analyze research findings',
  backstory='A meticulous analyst with a knack for uncovering patterns'
)
writer = Agent(
  role='Writer',
  goal='Draft the final report',
  backstory='A skilled writer with a talent for crafting compelling narratives'
)

# Define the tasks in sequence
research_task = Task(description='Gather data on black holes', agent=researcher,expected_output='A comprehensive 3 paragraphs long report on black holes.',
  tools=[search_tool])
analysis_task = Task(description='Analyze the data to get any latest trends', agent=analyst,expected_output='A 2 sentence analysis on data based on the longer report.',
  tools=[search_tool],)
writing_task = Task(description='Compose the report', agent=writer,expected_output='A comprehensive page long report on black holes with data to support.',
  tools=[search_tool])

# Form the crew with a sequential process
report_crew = Crew(
  agents=[researcher, analyst, writer],
  tasks=[research_task, analysis_task, writing_task],
  process=Process.sequential
)

2024-04-03 11:18:44,951 - 8095669312 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [23]:
# Starting the task execution process with enhanced feedback
result = report_crew.kickoff(inputs={'topic': 'black holes'})
print(result)

 


Search results: Title: Black Holes - NASA Science
Link: https://science.nasa.gov/universe/black-holes/
Snippet: These objects aren't really holes. They're huge concentrations of matter packed into very tiny spaces. A black hole is so dense that gravity just beneath its ...
---
Title: Black hole - Wikipedia
Link: https://en.wikipedia.org/wiki/Black_hole
Snippet: A black hole is a region of spacetime ; Objects whose gravitational fields ; Black holes of stellar mass form when massive stars collapse at the end of their life ...
---
Title: Exploring Black Holes | NSF - National Science Foundation
Link: https://new.nsf.gov/blackholes
Snippet: Most black holes are the condensed remnants of a massive star, the collapsed core that remains following an explosive supernova. However, the black hole family ...
---
Title: Black Holes | Center for Astrophysics | Harvard & Smithsonian
Link: https://www.cfa.harvard.edu/research/topic/black-holes
Snippet: Black holes are some of the most fascinatin

In [24]:
# Define your agents with roles, goals, and tools
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory=(
    "You are a Senior Research Analyst at a leading tech think tank."
    "Your expertise lies in identifying emerging trends and technologies in AI and data science."
    "You have a knack for dissecting complex data and presenting actionable insights."
  ),
  verbose=True,
  allow_delegation=False,
  tools=[search_tool]
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory=(
    "You are a renowned Tech Content Strategist, known for your insightful and engaging articles on technology and innovation."
    "With a deep understanding of the tech industry, you transform complex concepts into compelling narratives."
  ),
  verbose=True,
  allow_delegation=True
)

# Create tasks for your agents
task1 = Task(
  description=(
    "Conduct a comprehensive analysis of the latest advancements in AI in 2024."
    "Identify key trends, breakthrough technologies, and potential industry impacts."
    "Compile your findings in a detailed report."
    "Make sure to check with a human if the draft is good before finalizing your answer."
  ),
  expected_output='A comprehensive full report on the latest AI advancements in 2024, leave nothing out',
  agent=researcher,
  human_input=True, # setting the flag on for human input in this task
)

task2 = Task(
  description=(
    "Using the insights from the researcher's report, develop an engaging blog post that highlights the most significant AI advancements."
    "Your post should be informative yet accessible, catering to a tech-savvy audience."
    "Aim for a narrative that captures the essence of these breakthroughs and their implications for the future."
  ),
  expected_output='A compelling 3 paragraphs blog post formatted as markdown about the latest AI advancements in 2024',
  agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-04-03 11:41:06,126 - 8095669312 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Research Analyst
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.Identify key trends, breakthrough technologies, and potential industry impacts.Compile your findings in a detailed report.Make sure to check with a human if the draft is good before finalizing your answer.


> Entering new CrewAgentExecutor chain...
Given the task at hand, I should first conduct thorough research on the latest advancements in AI in 2024. I will use the 'Search the internet' tool to fetch the most recent data and information on the subject matter.

Action: Search the internet
Action Input: {"search_query": "latest advancements in AI in 2024"} 


Search results: Title: The most important AI trends in 2024 - IBM Blog
Link: https://www.ibm.com/blog/artificial-intelligence-trends/
Snippet: The most important AI trends in 2024 · Reality check: more realistic expectations · Multimodal AI · Small(er) language models and open s

In [50]:
from crewai_tools import PDFSearchTool

# Initialize the tool with a specific PDF path for exclusive search within that document
tool = PDFSearchTool(pdf='Rocket Mortgage.pdf')

In [51]:
# Define your agents with roles, goals, and tools
researcher = Agent(
  role='Ratesheet Analyzer',
  goal='Analyze ratesheet to derive keep insights',
  backstory=(
    "You are an experience loan officer who understands mortgage terms and can understand rate sheets"
    "Your expertise lies in providing the correct rate given a borrower's situation."
    "You have a knack for dissecting complex data many in the form of tables and presenting actionable insights."
  ),
  verbose=True,
  allow_delegation=False,
  tools=[tool]
)

# Create tasks for your agents
task1 = Task(
  description=(
    "Find 30 year conventional loan rate with a 15 day lock for a person with a 700 credit score and $100,000 annual income with no debts, looking for a single family home in San Francisco with loan-to-value of 20%."
  ),
  expected_output='The rate for the given loan product and borrower.',
  agent=researcher,
  human_input=True, # setting the flag on for human input in this task
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher],
  tasks=[task1],
  verbose=2
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-04-04 00:00:37,856 - 8095669312 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Ratesheet Analyzer
 [INFO]: == Starting Task: Find 30 year conventional loan rate with a 15 day lock for a person with a 700 credit score and $100,000 annual income with no debts, looking for a single family home in San Francisco with loan-to-value of 20%.


> Entering new CrewAgentExecutor chain...
I need to find the interest rate for a 30-year conventional loan with a 15-day lock period. The borrower has a credit score of 700, an annual income of $100,000, no debts, is looking to buy a single-family home in San Francisco and the loan-to-value is 20%. The first step is to use the tool to search for the relevant information in the PDF.

Action: 
Search a PDF's content

Action Input: 
{
"search_query": "30 year conventional loan 15 day lock 700 credit score"
}
Doc content has not changed. Skipping creating chunks and embeddings
 

Relevant Content:
0.000Subordinate financing DTI Ratio > 40% Manufactured Housing is Available Please Price Online For BPMI and LP